# Ovde ide dobijeni beat

In [78]:
tempo = 120

In [79]:
# Primer beat-a
beat = [1/4, 1/8, 1/8, 1/4, 1/8, 1/8, 1/4, 1/8, 1/8, 1/2]

# Potrebna muzicka teorija

In [80]:
def note_duration(note_length):
    bar_duration = 60 / tempo * 4
    return note_length * bar_duration

In [81]:
# Keys
#              0    1     2    3     4    5     6    7    8     9    10    11
piano_keys = ['c', 'cs', 'd', 'ds', 'e', 'f', 'fs', 'g', 'gs', 'a', 'as', 'b']

# Modes
modes = {
    "major" : [0, 2, 4, 5, 7, 9, 11],
    "minor" : [0, 2, 3, 5, 7, 8, 10]
}

In [82]:
# Note u skali
def scale(root, mode):
    scl = []
    n = piano_keys.index(root)

    for i in modes[mode]:
        scl.append(piano_keys[(i+n)%12])

    return scl

In [83]:
for mode in modes:
    for key in piano_keys:
        print(key + " " + mode + ": ", scale(key, mode))

c major:  ['c', 'd', 'e', 'f', 'g', 'a', 'b']
cs major:  ['cs', 'ds', 'f', 'fs', 'gs', 'as', 'c']
d major:  ['d', 'e', 'fs', 'g', 'a', 'b', 'cs']
ds major:  ['ds', 'f', 'g', 'gs', 'as', 'c', 'd']
e major:  ['e', 'fs', 'gs', 'a', 'b', 'cs', 'ds']
f major:  ['f', 'g', 'a', 'as', 'c', 'd', 'e']
fs major:  ['fs', 'gs', 'as', 'b', 'cs', 'ds', 'f']
g major:  ['g', 'a', 'b', 'c', 'd', 'e', 'fs']
gs major:  ['gs', 'as', 'c', 'cs', 'ds', 'f', 'g']
a major:  ['a', 'b', 'cs', 'd', 'e', 'fs', 'gs']
as major:  ['as', 'c', 'd', 'ds', 'f', 'g', 'a']
b major:  ['b', 'cs', 'ds', 'e', 'fs', 'gs', 'as']
c minor:  ['c', 'd', 'ds', 'f', 'g', 'gs', 'as']
cs minor:  ['cs', 'ds', 'e', 'fs', 'gs', 'a', 'b']
d minor:  ['d', 'e', 'f', 'g', 'a', 'as', 'c']
ds minor:  ['ds', 'f', 'fs', 'gs', 'as', 'b', 'cs']
e minor:  ['e', 'fs', 'g', 'a', 'b', 'c', 'd']
f minor:  ['f', 'g', 'gs', 'as', 'c', 'cs', 'ds']
fs minor:  ['fs', 'gs', 'a', 'b', 'cs', 'd', 'e']
g minor:  ['g', 'a', 'as', 'c', 'd', 'ds', 'f']
gs minor:  ['g

# Sviranje dobijene melodije

In [84]:
import pygame as pg
import time
from threading import Thread

In [85]:
pg.mixer.init()
pg.init()

(5, 0)

In [86]:
def path(note):
    return "Piano/" + note + ".wav"

In [87]:
def play_note(note, duration):
    pg.mixer.music.load(path(note))
    pg.mixer.music.play()
    time.sleep(note_duration(duration))

# Klasa Melody

In [88]:
import random

In [89]:
class Melody:

    id = 1

    def __init__(self, beat, scale_root, scale_mode):
        self.id = Melody.id
        Melody.id += 1
        self.scale_root = scale_root
        self.scale_mode = scale_mode
        self.notes_in_scale = scale(self.scale_root, self.scale_mode)
        self.notes = self.generate_melody(beat)
        self.fitness = self.calc_fitness()

    def generate_melody(self, beat):
        notes = []
        for i in range(len(beat)):
            octave = random.randint(3,6)
            note = note = random.choice(self.notes_in_scale)
            note = note + str(octave)
            notes.append([note, beat[i]])
        return notes

    def print(self):
        for note in self.notes:
            print(note[0], end=" ")
        print()

    def play(self):
        print("Melody " + str(self.id))
        pg.mixer.set_num_channels(len(self.notes))
        th = {}
        n = 0
        for i in self.notes:
            th[n] = Thread(target=play_note, args=(i[0], i[1]))
            th[n].start()
            th[n].join()
            n += 1

    def calc_fitness(self):
        self.play()
        rating = int(input("Rating: "))
        print()
        return rating

# Genetic Algorithm

In [90]:
def selection(population, tournament_size):
    pass

In [91]:
def crossover(parent1, parent2, child1, child2):
    pass

In [92]:
def mutation(melody, mutation_prob):
    pass

In [93]:
def genetic_algorithm(
    num_bars=8, 
    scale_root="c", 
    scale_mode="major", 
    population_size=4, 
    num_generations=4, 
    elitism_size=2, 
    tournament_size=2, 
    mutation_probability=0.5
):

    genId = 1
    print("GENERATION " + str(genId))
    print()
    population = [Melody(beat, scale_root, scale_mode) for _ in range(population_size)]
    new_population = population.copy()

    while True:

        if input("Next generation? [y/n]: ") == 'n':
            break

        genId += 1
        print("GENERATION " + str(genId))
        print()
        
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = population[:elitism_size]
        
        for j in range(elitism_size, population_size, 2):
            
            parent1 = selection(population, tournament_size)
            parent2 = selection(population, tournament_size)
            
            crossover(parent1, parent2, child1=new_population[j], child2=new_population[j+1])
    
            mutation(new_population[j], mutation_probability)
            mutation(new_population[j+1], mutation_probability)
            
            new_population[j].fitness = new_population[j].calc_fitness()
            new_population[j+1].fitness = new_population[j+1].calc_fitness()
    
        population = new_population.copy()

    return max(population, key=lambda x: x.fitness)

# Main

In [94]:
melody = genetic_algorithm()

GENERATION 1

Melody 1


Rating:  2



Melody 2


Rating:  2



Melody 3


...

GENERATION 4

Melody 3


Rating:  2



Melody 4


Rating:  1


Next generation? [y/n]:  n


In [95]:
melody.print()

f4 f6 e4 a5 c6 c3 f6 c6 f5 e5 
